**Prod Version of DevEnv**

**This notebook is run using nbconvert after devenv service starts**

In [1]:
!pip install twitter
!pip install feedparser
!pip install schedule
!pip install kafka-python
!pip install redis

     |████████████████████████████████| 61kB 791kB/s eta 0:00:011
     |████████████████████████████████| 194kB 1.2MB/s eta 0:00:01
  Created wheel for feedparser: filename=feedparser-5.2.1-cp37-none-any.whl size=44940 sha256=f77314c234badb8325fd396e9f479a5269b998a1ffe62d5897bd602c051027eb
  Stored in directory: /home/jovyan/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser
     |████████████████████████████████| 266kB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.4MB/s eta 0:00:01


In [2]:
from time import sleep
from json import dumps, loads, load
from kafka import KafkaProducer
from datetime import datetime, timedelta
from feedparser import parse
from urllib.request import urlopen
from twitter import Twitter
import schedule
from datetime import datetime

In [3]:
tickers = ["MSFT","AAPL"]
feeds = ["http://feeds.reuters.com/reuters/businessNews"]

producer = KafkaProducer(
    bootstrap_servers=['broker:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

In [4]:
def DownloadStreams():
    # Yahoo Finance Stream for Stock Price
    f_data = {}
    for ticker in tickers:
        try:
            y_finance = load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/"+ticker+"?range=1h&interval=1h"))
            meta = y_finance['chart']['result'][0]['meta']
            timestamp = meta['regularMarketTime']
            price = meta['regularMarketPrice']
            f_data[ticker] = {"Type": "Finance", "Ticker": ticker, "Timestamp":timestamp, "Price":price}
        except:
            print("Exception in Download Stream for "+ticker)
    
    # News Stream from Reuters for Business
    feed_data = {}
    for feed in feeds:
        feed_data[feed] = [{"Type": "News", "Timestamp": datetime.timestamp(datetime.now()), "Title":l.title, "Summary":l.summary, "Link":l.link} for l in (parse(feed))["entries"]]
        # print([l.title for l in d["entries"]])

    return f_data, feed_data

def UploadStreamsToKafka(f_data, feed_data):
    for key in f_data.keys():
        producer.send('finance_ticker', value=f_data[key])
    for key in feed_data.keys():
        for entry in feed_data[key]:
            producer.send('news_feed', value=entry)
    producer.flush()

def HourlyJob():
    # Check, Download & Parse each stream
    f_data, feed_data = DownloadStreams()
    # Upload cleaned events into Kafka Topics
    UploadStreamsToKafka(f_data, feed_data)
    print("Hourly Run Events Complete - Sleeping now")

In [5]:
HourlyJob()

Hourly Run Events Complete - Sleeping now


In [ ]:
schedule.clear()
HourlyJob()

schedule.every(15).minutes.do(HourlyJob)
starttime = datetime.now()
endtime = starttime + timedelta(minutes=5)

while True: #datetime.now()<endtime:
    schedule.run_pending()
    sleep(10)
    
schedule.clear()

Hourly Run Events Complete - Sleeping now
